In [1]:
# Importation of libraries
import os
import sys
import scipy
import trimesh
import numpy as np
import open3d as o3d
from PIL import Image
import matplotlib as mpl
import matplotlib.cm as cm
from open3d import JVisualizer

# Loading a mesh

In [2]:
# Disable Print function
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore Print function
def enablePrint():
    sys.stdout = sys.__stdout__

In [3]:
def load_obj(file_path):
    with open(file_path, 'r') as file_:
        lines = file_.readlines()
    vertices = []
    vertex_textures = []
    face_vertices = []
    face_textures = []
    for line in lines:
        if line.split(' ')[0] == 'v':
            vertices.append(list(map(float, line.replace('\n','').split(' ')[1:])))
        elif line.split(' ')[0] == 'vt':
            vertex_textures.append(list(map(float, line.replace('\n','').split(' ')[1:])))
        elif line.split(' ')[0] == 'f':
            face_vertices.append(list(map(int, [elt.split('/')[0] for elt in line.replace('\n','').split(' ')[1:] if elt])))
            face_textures.append(list(map(int, [elt.split('/')[1] for elt in line.replace('\n','').split(' ')[1:] if elt])))
    vertices = np.array(vertices)
    vertex_textures = np.array(vertex_textures)
    face_vertex_indices = np.array(face_vertices) - 1 # Because indexing in Python starts at 0
    face_textures = np.array(face_textures) - 1 # Because indexing in Python starts at 0
    face_textures = np.reshape(vertex_textures[np.reshape(face_textures, [-1])], [-1, 3, 2])
    return vertices, face_vertex_indices, face_textures

In [4]:
def get_per_vertex_texture(face_vertex_indices, face_textures):
    _, _, dim = face_textures.shape
    unique_values, unique_indices = np.unique(face_vertex_indices.flatten(), return_index=True)
    assert(np.all(unique_values == np.arange(len(unique_values))))
    vertex_textures = np.reshape(face_textures, [-1, dim])[unique_indices]
    return vertex_textures

In [5]:
def show_mesh(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh])

In [6]:
# Path to the files
path_source = 'source_data'
path_output = 'export_data'
if not os.path.isdir(path_output):
    os.mkdir(path_output)
mesh_file = os.path.join(path_source, 'mesh.obj')
texture_file = os.path.join(path_source, 'mesh.png')
show_mesh(mesh_file)

<img src="images/screenshot1.png">

In [7]:
# Processing
vertices, face_vertex_indices, face_textures = load_obj(mesh_file)
num_points, _ = vertices.shape
vertex_textures = get_per_vertex_texture(face_vertex_indices, face_textures)

# Showing the euclidean distance as a vertex function on a mesh

In [8]:
def get_euclidean_distance(vertices, index):
    distances = np.linalg.norm(vertices - vertices[index], axis=1)
    return distances

In [9]:
def get_color_mapping(input_values, cmap):
    norm = mpl.colors.Normalize(vmin=distances.min(), vmax=distances.max())
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    colors = m.to_rgba(input_values)
    return colors

In [10]:
index_point = np.random.randint(0, num_points)
distances = get_euclidean_distance(vertices, index_point)
vertex_colors_mesh = get_color_mapping(distances, cm.hot)
new_mesh = trimesh.Trimesh(vertices=vertices, faces=face_vertex_indices, vertex_colors=vertex_colors_mesh)

sphere = trimesh.creation.uv_sphere(radius=0.05, count=[32, 32])
sphere.vertices = sphere.vertices + vertices[index_point]
vertex_colors_sphere = np.zeros([sphere.vertices.shape[0], 4])
vertex_colors_sphere[:,1] = 1
sphere.visual.vertex_colors = vertex_colors_sphere
new_mesh = trimesh.util.concatenate(new_mesh, sphere)
vertex_colors = np.concatenate((vertex_colors_mesh, vertex_colors_sphere), axis=0)

blockPrint()
new_mesh.export(os.path.join(path_output,'Euclidean_distances.ply'))
enablePrint()
show_mesh(os.path.join(path_output,'Euclidean_distances.ply'))

<img src="images/screenshot2.png">

# UV mapping on a mesh

In [11]:
vertex_colors = np.concatenate((vertex_textures, np.zeros_like(vertex_textures[:,:1])), axis=1)
new_mesh = trimesh.Trimesh(vertices=vertices, faces=face_vertex_indices, vertex_colors=vertex_colors)
blockPrint()
new_mesh.export(os.path.join(path_output,'UV_map.ply'))
enablePrint()
show_mesh(os.path.join(path_output,'UV_map.ply'))

<img src="images/screenshot3.png">

# Showing the texture

In [12]:
def apply_texture(face_textures, texture_file):
    texture = np.asarray(Image.open(texture_file))
    texture = np.flip(texture, axis=0)
    texture = texture.transpose(1,0,2)

    size_a, size_b, _ = texture.shape
    face_textures_int = face_textures.copy()
    face_textures_int[:,:,0] *= size_a
    face_textures_int[:,:,1] *= size_b
    face_textures_int = np.reshape(np.round(face_textures_int).astype(int), [-1, 2])
    face_textures_int = np.reshape(texture[face_textures_int[:,0],face_textures_int[:,1],:], [-1, 3, 3])
    return face_textures_int

In [13]:
face_textures_int = apply_texture(face_textures, texture_file)
vertex_colors = get_per_vertex_texture(face_vertex_indices, face_textures_int)

In [14]:
new_mesh = trimesh.Trimesh(vertices=vertices, faces=face_vertex_indices, vertex_colors=vertex_colors)
blockPrint()
new_mesh.export(os.path.join(path_output,'Texture.ply'))
enablePrint()
show_mesh(os.path.join(path_output,'Texture.ply'))

<img src="images/screenshot4.png">

# Getting the different rings (vertex neighbourhood)

In [15]:
def get_adjacency_matrix(vertices, faces):
    num_points, _ = vertices.shape
    row0 = np.concatenate((faces[:,0], faces[:,0], faces[:,0]), axis=0)
    col0 = np.concatenate((faces[:,0], faces[:,1], faces[:,2]), axis=0)
    row1 = np.concatenate((faces[:,1], faces[:,1], faces[:,1]), axis=0)
    col1 = np.concatenate((faces[:,0], faces[:,1], faces[:,2]), axis=0)
    row2 = np.concatenate((faces[:,2], faces[:,2], faces[:,2]), axis=0)
    col2 = np.concatenate((faces[:,0], faces[:,1], faces[:,2]), axis=0)
    row = np.concatenate((row0, row1, row2), axis=0)
    col = np.concatenate((col0, col1, col2), axis=0)
    data = np.ones_like(row)
    adjacency_matrix = scipy.sparse.coo_matrix((data, (row, col)), shape=(num_points, num_points))
    return adjacency_matrix

In [16]:
num_points, _ = vertices.shape
adjacency_matrix = get_adjacency_matrix(vertices, face_vertex_indices)

In [17]:
first_ring = adjacency_matrix
second_ring = adjacency_matrix @ first_ring
third_ring = adjacency_matrix @ second_ring

In [18]:
scene = [new_mesh]
index_point = np.random.randint(0, num_points)
sphere = trimesh.creation.uv_sphere(radius=0.03, count=[32, 32])
sphere.vertices = sphere.vertices + vertices[index_point]
num_vertices_sphere, _ = sphere.vertices.shape
vertex_color = np.zeros([num_vertices_sphere, 4])
vertex_color[:,0] = 1
sphere.visual.vertex_color = vertex_color
scene.append(sphere)
for index in second_ring[index_point].indices:
    sphere = trimesh.creation.uv_sphere(radius=0.01, count=[32, 32])
    sphere.vertices = sphere.vertices + vertices[index]
    vertex_color = np.zeros([num_vertices_sphere, 4])
    vertex_color[:,1] = 1
    sphere.visual.vertex_color = vertex_color
    scene.append(sphere)
mesh = trimesh.util.concatenate(scene)
blockPrint()
mesh.export(os.path.join(path_output,'Neighbourhood.ply'))
enablePrint()
show_mesh(os.path.join(path_output,'Neighbourhood.ply'))

<img src="images/screenshot5.png">